---
Lab3:Genotype Imputation with IMPUTE2
---
**Before getting started, here are some notes on how to manage softwares, data, and output on cluster**. 
It is better to to have data, software, and files organized into folders rather than putting everything at one place.
And thoughout this lab, you can maintain a folder stucture like:

In [3]:
tree lab3/ -L 2

lab3/  [error opening dir]

0 directories, 0 files


: 2

Genotype imputation relies on statistical inference and known haplotypes in a population to estimate unobserved genotypes. Often, particularly in epidemiology and quantitative genetics, a researcher may want to identify a specific location in the genome where genetic variation is associated with a particular phenotype, but they may only have access to sparse genotype data (e.g. data from a SNP genotyping array as opposed to complete sequence/whole genome sequencing data). At the same time, dense genotype data may exist for a reference panel of individuals (e.g. from the HapMap project or 1000 Genomes for humans). By leveraging the information in the reference panel to impute gentoypes into the inference panel, a reseracher can interrogate more variants for association with the trait.  In some cases, this reveals associations that would not be detected directly in the sparse data. In other cases, it may help refine the location of a causal variant. 

The imputation process relies on haplotype phasing, which is the statistical estimation of haplotypes from genotype data. Both of these steps can be accomplished from within IMPUTE2, and will be the focus of today's lab. By the end of the lab you should be able to:

- **Understand basic workflow and syntax of IMPUTE2**  
- **Execute basic imputation in IMPUTE2**
- **Navigate basic commands for file conversion using GTOOL**  
- **Understand key assumptions of imputation**
- **Utilize IMPUTE2 and PLINK to search for putative associations in imputed SNPs**

# Basics of IMPUTE2

## BASH commands useful for this lab 

Command     |     Description
----------  |   ----------------------------------
**cut**     |   Extract sections from each line of input stream (i.e. file).  
**sed**     |   Stream editor for performing basic text transformations on an input stream.
**paste**   |   Merges lines of files together.
**wc**      |   Word, line, character, and byte count.  
**mkdir**   |   Make directory.
**head**    |   Output first part of files.
**grep**    |   Prints lines matching a pattern (helpful for manipulating large files).

## Standard IMPUTE2 arguments

Argument      |  Input Type     |       Description
-----------   | -------         |     -----------------------------------
**-g**        |   .gens         |   File containing study genotypes needing imputation or phasing.  
**-m**        |   .map          |   Fine-scale recombination map for the region to be analyzed.
**-int**      |   [*interval*] |  Genomic interval to use for imputation inference.
**-h**        |   .haps         |  File of known haplotypes, with one row per SNP and one column per haplotype (alleles must be coded 0 or 1)
**-l**        |  .legend        |   Legend file(s) with information about the SNPs in the haplotypes file. 
**-strand_g**  |   .strand       |   File containing strand orientation of the inference set. 
**-Ne**        |   [*pop size*]  |   Parameter controlling effective population size. 


## Downloading and Running IMPUTE2

**Follow the command below to download IMPUTE2 and go to the Example directory**:

In [4]:
#mkdir IMPUTE2
cd IMPUTE2
#curl https://mathgen.stats.ox.ac.uk/impute/impute_v2.3.2_x86_64_static.tgz > impute2.tgz  
#tar -zxvf impute2.tgz
cd impute_v2.3.2_x86_64_static/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1464k  100 1464k    0     0  12772      0  0:01:57  0:01:57 --:--:--  9607
impute_v2.3.2_x86_64_static/
impute_v2.3.2_x86_64_static/Example/
impute_v2.3.2_x86_64_static/Example/example.chr22.1kG.annot.legend
impute_v2.3.2_x86_64_static/Example/example.chr22.1kG.haps
impute_v2.3.2_x86_64_static/Example/example.chr22.1kG.legend
impute_v2.3.2_x86_64_static/Example/example.chr22.hm3.haps
impute_v2.3.2_x86_64_static/Example/example.chr22.hm3.legend
impute_v2.3.2_x86_64_static/Example/example.chr22.map
impute_v2.3.2_x86_64_static/Example/example.chr22.one.phased.impute2
impute_v2.3.2_x86_64_static/Example/example.chr22.one.phased.impute2_info
impute_v2.3.2_x86_64_static/Example/example.chr22.one.phased.impute2_info_by_sample
impute_v2.3.2_x86_64_static/Example/example.chr22.one.phased.impute2_summary
impute_v2.3.2_x86_64_static/Exam

In [8]:
ls

Example  impute2  LICENCE


## File formatting for IMPUTE2  

There are several file formats necessary for IMPUTE2, which are listed above in Section 1.2. It is helpful to understand the basic formatting for these input files and their importance in the imputation, as this will facilitate manipulating them in the future. 

### GENS file

The GENS file is the general file type for genotype data in IMPUTE2. Each row represents a different SNP, and the first five columns are: (1) **SNP ID**, (2) **rsID**, (3) **Position (bp)**, (4) **Major Allele**, (5) **Minor Allele**. The sixth column onward represents the genotypic information for all of the individuals, where each individual has three columns representing the probabilities of each genotype. This file does NOT have a header row.

### MAP file  

The MAP file provides a fine-scale recombination map with three columns: (1) *physical position (in base pairs)*, (2) *recombination rate between current position and next position in map (in cM/Mb)*, (3) *and genetic map position (in cM)*. This information is critical in the process of imputation in that altering rates of recombination in a region will significantly change the likelihoods for imputed SNPs. For example, if there are high rates of recombination between SNP A from your SNP array and desired imputed SNP B, you may have less statistical power or certainty to impute that SNP depending on the depth of your reference haplotypes. Note that this file DOES have a header row. You can always check if a file has one (and you should need to do it for a few times in this lab) simply by opening it with your text editor of choice.

*Problem 1*  
All the files that begin with the label "example.chr22" are from a particular stretch of DNA on chromosome. What position in this stretch has the highest recombination rate? What is it (please include units)?

### HAPS file  

The HAPS file is a file of reference haplotypes (usually binary), often from a larger project (i.e. 1000 Genomes), which is used during imputation to compare against haplotypes from the inference panel to estimate probabilities for imputed SNPs. Each column represents a distinct haplotype to be used as a reference.  

*Problem 2*  
How many haplotypes are provided in the example file *example.chr22.hm3.haps*? How many in the example file *example.chr22.1kG.haps*? (hint: one way to look at the number of columns is through `awk '{print NF}'` where `NF` stands for the number of fields)

### LEGEND file  

The LEGEND file provides information on the HAPS file, so there must be a one-to-one correspondence. Thus, it is generally not wise to edit the HAPS or the LEGEND file, but if you were to do this you would need to edit them in concert. Each LEGEND file provides four columns of information, with each row corresponding to the same row in the HAPS file. The four columns of information are: (1) **rs ID**, (2) **Position (bp)**, (3) **Major Allele**, (4) **Minor Allele**. 

*Problem 3*  
What is the major allele for SNP "rs1669115"? What is the minor allele? (hint: `grep` command could be useful)

### STRAND file  

The STRAND file is a pretty simple but critical file that ensures the reference set and the inference set are comparing the same strand of DNA (similar 5' to 3' distinction). This avoids confusion about ambigous genotype calls -- for example, what one investigator calls a GG call could be called a CC by another investigator looking at the opposing strand of DNA. To denote which strand we use relative to the reference panel, a "+" denotes the same strand as the reference panel, and a "-" denotes the opposing strand. There are only two columns: (1) **Position (bp)**, (2) **Strand**. You really shouldn't be altering this file unless completely necessary, as it can introduce some very big errors. Although this won't be necessary for today's lab, it is essential to have the strand matched between the reference and imputation panel when imputing data and one should be on guard for "strand flips" as an explanation for anomolous results.

## Running the imputation {#sec:impute}

For getting the hang of running IMPUTE2, we will be imputing the area on the 22nd chromosome between the SNPs sequenced in the example files *"example.chr.study"*. These files specify the necessary information for our **inference panel**. Likewise, we have a **reference panel** in this same region specified in the files starting with *"example.chr.1kG"*.

In [9]:
# in the directory where impute2 executable locates
./impute2 \
  -m ./Example/example.chr22.map \
  -h ./Example/example.chr22.1kG.haps \
  -l ./Example/example.chr22.1kG.legend \
  -g ./Example/example.chr22.study.gens \
  -strand_g ./Example/example.chr22.study.strand \
  -int 20.3e6 20.6e6 \
  -Ne 20000 \
  -o ../../output/impute2_output-example.chr22.one.phased.impute2 #remember to create a folder called 'output' in your lab3 directory



 IMPUTE version 2.3.2 

Copyright 2008 Bryan Howie, Peter Donnelly, and Jonathan Marchini
Please see the LICENCE file included with this program for conditions of use.

The seed for the random number generator is 48101716.

Command-line input: ./impute2 -m ./Example/example.chr22.map -h ./Example/example.chr22.1kG.haps -l ./Example/example.chr22.1kG.legend -g ./Example/example.chr22.study.gens -strand_g ./Example/example.chr22.study.strand -int 20.3e6 20.6e6 -Ne 20000 -o ../../output/impute2_output-example.chr22.one.phased.impute2

---------------------------------
 Nomenclature and data structure 
---------------------------------

     Panel 0: phased reference haplotypes
     Panel 2: unphased study genotypes

For optimal results, each successive panel (0,1,2) should contain a subset of the SNPs in the previous panel. When the data structure deviates from this ideal configuration, IMPUTE2 tries to use as much of the available information as possible; see documentation for details.


Note that you will need to be in the directory where you have unpacked the impute2 tarball, and where you can see the *Example* directory listed out, in order for this call to work. 

This should create five files in the specified output directory, all of which should start with *"impute2_output-example.chr22.one.phased.impute2"*. The file with the actual imputed genotypes in GENS format should be the file *just* called *"impute2_output-example.chr22.one.phased.impute2"*. Other supplementary output files have underscores and then IDs: **"warning"** gives any errors during the imputation process, **"summary"** is just a log file, **"info"** gives a variety of information for each SNP as demonstrated in the header, and **"info by sample"** gives information by individuals instead of by SNP. 

*Problem 4*  
Open the summary file in a text editor and scroll down to line 60 (L60), where it should mention 'flipping strands'. What does it say and how would you interpret this? 

## Using GTOOL post-imputation

GTOOL is a command-line program that facilitates conversion between PLINK and IMPUTE2 file formats. The program is well documented, and it is recommended that you visit the documentation on the website for any purposes outside this lab (http://www.well.ox.ac.uk/~cfreeman/software/gwas/gtool.html).

### Run GTOOL 

Go back to your *software* directory and **use the following command to Run GTOOL**:

In [61]:
#mkdir GTOOL
cd ../../GTOOL
ls
chmod a+x ./gtool 

example  gtool  gtool.log  LICENCE


### Basic GTOOL usage {#sec:gtool}

Here we will practice using GTOOL on one of the small GENS files with only 33 markers from the IMPUTE2 *"Example/"* directory, *"example.chr22.study.gens"*. We will need to provide this file, as well as a list of samples "example.study.samples" in the same directory.

In [62]:
# in the directory where gtool executable locates
./gtool \
  -G \
  --g ../IMPUTE2/impute_v2.3.2_x86_64_static/Example/example.chr22.study.gens \
  --s ../IMPUTE2/impute_v2.3.2_x86_64_static/Example/example.study.samples \
  --ped ../../output/gtool_output-real_small.ped \
  --map ../../output/gtool_output-real_small.map \
  --threshold 0.9

./gtool -G --g ../IMPUTE2/impute_v2.3.2_x86_64_static/Example/example.chr22.study.gens --s ../IMPUTE2/impute_v2.3.2_x86_64_static/Example/example.study.samples --ped ../../output/gtool_output-real_small.ped --map ../../output/gtool_output-real_small.map --threshold 0.9
Count SNPs and samples...
Number of samples: 250
Number of SNPs: 33
Sample...
Assign memory for genotypes...16500
Gen...
Number of missing loci: 65
Output...


Don't forget that you will need to modify this command to find your own gtool executable. You may also need to modify the paths to the example.chr22 and example.study.samples files (These are example files from IMPUTE2), based on where those are in your workspace. 

Before running your GTOOL command, take a minute to look at the arguments of gtool. One important parameter is the **"--threshold"** argument. This tells GTOOL the likelihood cut-off for a genotype at any given SNP. For example, in our command, any likelihood below 0.9 at a given SNP will produce a "missing genotype" for PLINK. 

*Problem 5*  
Recalling your knowledge of the structure of PED files, what do you notice about both the phenotype and gender information in the file we just converted ("gtool_output-real_small.ped")?

# Run a Case-Control GWAS with/without Imputation

In this section we will use the skills we have learned in the first section to simulate a scenario in which you have localized a region through previous work on the 22nd chromosome that is associated with Disease XXX. You will need to use commands from PLINK, GTOOL, and IMPUTE2 to work through these scenarios (consult previous labs if you have trouble remembering how to run certain code), in which you will run an association, identify the SNP on your array, convert data to IMPUTE2 format, and impute unobserved SNPs for another association in an attempt to get a more precise estimate of the associated SNP.

To give you an overview, suppose our collaborator give you genotype data (files shared with IMPUTE2 software) along with phenotype file (`phenotype.txt` in shared data). 
We'd like to run GWAS with the raw genotypes and the imputed genotypes.
And see how they could be different from each other.

## Running an initial GWAS

Note that to run GWAS using PLINK, we first convert the genotype into PED format using GTOOL. This step has been done by [this section](#sec:gtool). Here, you will run an initial GWAS on a set of 33 markers in the file *"gtool_output-real_small"*. For the sake of this exercise, assume that these 33 markers restricted to a small section of chromosome 22 (20303319-20596808bp) are part of a larger GWAS that you ran with a total of 100,000 SNPs. In that study, you identified this region, and now you want to pursue your study further with imputation. Just to check that everything is working correctly, you want to check the reduced set (33 markers) with a GWAS.

*Problem 6*  
Run a basic association (--assoc) on the *"gtool_output-real_small"* dataset (it should be the output generated in [this section](#sec:gtool)) and name the resulting files with *"gwas_raw"*. Show the commands you used. Hint: You may need the "--allow-no-sex" and "--pheno" flag. 

*Problem 7*  
According to this analysis, what is the most highly associated SNP? What is the unadjusted p-value? What do you think this SNP's adjusted (Bonferroni-correction) p-value should be?

## Manhattan plot of initial GWAS

*Problem 8*  
Use the knowledge gained in Lab 2 to export the *.assoc* file and produce a Manhattan plot in R. Make sure to include the Bonferroni-corrected significance threshold.

## Imputation and conversion

Imputation for this set of 33 markers was completed in [here](#sec:impute). The files you produced should be called *"impute2_output-example.chr22.one.phased.impute2"*. Let's take this GENS file and convert it to PED format.

*Problem 9*  
Using GTOOL, convert the file *"impute2_output-example.chr22.one.phased.impute2"* to PED format, and name the outputs with *"imputed-example.chr22.one.phased"*. 
Show your command.

## Running a GWAS with Imputed SNPs

*Problem 10*
Once you have your converted and reformatted PED and MAP files named *"imputed-example.chr22.one.phased"*, we can now run GWAS with imputed genotypes. Name the output files as *"gwas_imputed"*. Show your command.

*Problem 11*  
Find the most highly associated SNP in the *"gwas_imputed.assoc"* file. What SNP is it? What is the non-adjusted p-value? 

## Manhattan Plot of Imputed GWAS

Lastly, we will visualize the results of our GWAS with imputed SNPs by exporting our *.assoc"* file to our computer and plot it using R.

*Problem 12*  
Produce a Manhattan plot for the file *"gwas_imputed.assoc"* in R. Describe the differences between this plot and the plot from *"gwas_raw.assoc"* in a couple of sentences.